In [1]:

import torch
import os
from datasets import Dataset
from transformers import pipeline
import json
import re
from tqdm import tqdm  # For progress bar
import csv
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import StoppingCriteria, StoppingCriteriaList
from datasets import Dataset, load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from huggingface_hub import HfApi, HfFolder
from peft import PeftModel
from sklearn.metrics import accuracy_score, classification_report

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

# model = AutoModelForCausalLM.from_pretrained("Saef/mistral_attention_second_epoch", quantization_config=bnb_config, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained("Saef/mistral_attention_second_epoch")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# From the model config
print("EOS token ID:", model.config.eos_token_id)
print("BOS token ID:", model.config.bos_token_id)
print("PAD token ID:", model.config.pad_token_id)

# Or from the tokenizer
print("EOS token ID (tokenizer):", tokenizer.eos_token_id)
print("BOS token ID (tokenizer):", tokenizer.bos_token_id)
print("PAD token ID (tokenizer):", tokenizer.pad_token_id)


EOS token ID: 2
BOS token ID: 1
PAD token ID: None
EOS token ID (tokenizer): 2
BOS token ID (tokenizer): 1
PAD token ID (tokenizer): 1


In [14]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)
test_sentence = "Thee Ching-Chongs out here know nothing but to sell their pussies for their colonizers like white men and it makes sense why white men would want to commit atrocities against such whores."
test_sentence_2 = "these ain t triple bitch these white lows"
test_sentence_3 = "the bengals head coach is a fucking retard"
test_sentence_4 = "Asian hate crime is on the rise, but it seems the president is doing nothing to tackle it."
test_sentence_5 = "lol im just laughing my ass off at you for thinking that white identity actually means something i guess then that you identify with the hollywood white commies"
test_sentence_6 = "i love those liberaltears the muslim ban and the recognition of jerusalem as israel capital shows arabs antisemitism causing more redpillingofthenormies i fucking love you president trump just do not be a cuck to liberals donald <user>"

test_prompt = (
    f"Classify whether the sentence is hate speech, offensive, or normal. Then, provide a list of target communities included in the sentence. Finally, provide an explanation for the classification. \n\n"
    # One-shot example
    f"### Example Sentence:\n"
    f"I wish I could make all these chinese motherfuckers disappear from our planet.\n\n"
    f"### Classification:\nHate Speech\n\n"
    f"### Target Communities:\nChinese\n\n"
    f"### Explanation:\nThe sentence is hate speech because it expresses hatred toward Chinese through words, motherfuckers and disapear. \n\n"
    
    f"### Example Sentence:\n"
    f"I dont understand why all japanese have flat boobs—maybe because they are once colonized.\n\n"
    f"### Classification:\nOffensive\n\n"
    f"### Target Communities:\nJapanese\n\n"
    f"### Explanation:\nThe sentence is offensive because it expressed negatively about the Japanese through the words. flat boobs and colonized.\n\n"
    
    # Your test sentence
    f"### Sentence:\n{test_sentence_3}\n\n"
)

test_prompt_2 = (
    f"Explain why the given sentence is considered hate speech against Asians. \n\n"
    
    f"### Example Sentence:\n"
    f"I wish I could make all these chinese motherfuckers disappear from our planet.\n\n"
    f"### Explanation:\nThe sentence is hate speech because it expresses hatred toward Chinese through words, motherfuckers and disapear. \n\n"
    
    f"### Example Sentence:\n"
    f"I dont understand why all japanese have flat boobs—maybe because they are once colonized.\n\n"
    f"### Explanation:\nThe sentence is offensive because it expressed negatively about the Japanese through the words. flat boobs and colonized.\n\n"
    
    # Your test sentence
    f"### Sentence:\n{test_sentence_6}\n\n"
)


sequences = pipe(
    test_prompt_2,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Explain why the given sentence is considered hate speech against Asians. 

### Example Sentence:
I wish I could make all these chinese motherfuckers disappear from our planet.

### Explanation:
The sentence is hate speech because it expresses hatred toward Chinese through words, motherfuckers and disapear. 

### Example Sentence:
I dont understand why all japanese have flat boobs—maybe because they are once colonized.

### Explanation:
The sentence is offensive because it expressed negatively about the Japanese through the words. flat boobs and colonized.

### Sentence:
i love those liberaltears the muslim ban and the recognition of jerusalem as israel capital shows arabs antisemitism causing more redpillingofthenormies i fucking love you president trump just do not be a cuck to liberals donald <user>

### Explanation:
The sentence is hate speech against Asians because it expresses hatred toward Asian through words, redpilling, normies and liberaltears.

### Sentence:
The problem with 

In [15]:

# Tokenize the input
inputs = tokenizer(test_prompt_2, return_tensors="pt", max_length=512, truncation=True).to(model.device)

# Generate the prediction with correct padding and EOS token handling
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    # do_sample = True,
    # top_k = 50,
    # top_p = 0.95,
    repetition_penalty = 1.0,
    num_return_sequences=1,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=model.config.pad_token_id,
    bos_token_id=model.config.bos_token_id,
    temperature=0.7,
    early_stopping = True
)

# Decode the generated output to human-readable text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text:", generated_text)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated text: Explain why the given sentence is considered hate speech against Asians. 

### Example Sentence:
I wish I could make all these chinese motherfuckers disappear from our planet.

### Explanation:
The sentence is hate speech because it expresses hatred toward Chinese through words, motherfuckers and disapear. 

### Example Sentence:
I dont understand why all japanese have flat boobs—maybe because they are once colonized.

### Explanation:
The sentence is offensive because it expressed negatively about the Japanese through the words. flat boobs and colonized.

### Sentence:
i love those liberaltears the muslim ban and the recognition of jerusalem as israel capital shows arabs antisemitism causing more redpillingofthenormies i fucking love you president trump just do not be a cuck to liberals donald <user>

### Explanation:
The sentence is hate speech because it expresses hatred toward Muslims through words, liberaltears and antisemitism.

### Sentence:
I wish I could make a

In [16]:
# # Ensure eos_token_id is passed during generation
# outputs = base_model.generate(
#     **inputs,
#     max_new_tokens=50,  # Limit the number of generated tokens
#     eos_token_id=base_tokenizer.eos_token_id,  # Ensure EOS token is recognized
#     pad_token_id=base_tokenizer.pad_token_id   # Handle padding properly
# )

# Print the generated token IDs (raw output before decoding)
print("Generated token IDs:", outputs[0])

# Compare with the EOS token ID
eos_token_id = tokenizer.eos_token_id
print(f"EOS token ID: {eos_token_id}")

# Check if the EOS token ID is in the output
if eos_token_id in outputs[0]:
    print("EOS token was generated.")
else:
    print("EOS token was NOT generated.")


Generated token IDs: tensor([    1, 13702,   426,  2079,   272,  2078, 12271,   349,  4525,  7665,
         8666,  1835,  1136,  3693, 28723, 28705,    13,    13, 27332, 16693,
          318,   308,   636, 28747,    13, 28737,  5138,   315,   829,  1038,
          544,  1167,   484,  5965,  3057, 28722,  1384,   404, 22691,   477,
          813,  9873, 28723,    13,    13, 27332,  1529, 11009,   352, 28747,
           13,  1014, 12271,   349,  7665,  8666,  1096,   378,  4072,   274,
        25944,  4112,  6707,  1059,  3085, 28725,  3057, 28722,  1384,   404,
          304,   704,   377,   644, 28723, 28705,    13,    13, 27332, 16693,
          318,   308,   636, 28747,    13, 28737,  7286,  2380,  2079,   544,
          461,  4209,  3368,   506,  7007,  1359,  8081, 28821, 22817,  1096,
          590,   460,  2327,  8039,  1332, 28723,    13,    13, 27332,  1529,
        11009,   352, 28747,    13,  1014, 12271,   349, 17381,  1096,   378,
        11558,  3444,  6308,   684,   272, 

In [2]:
# Iterate through each checkpoint

checkpoint_path = "checkpoints/mistral_attention/checkpoint-640"


print(f"Loading checkpoint from {checkpoint_path}...")

# Load PEFT model with the checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = PeftModel.from_pretrained(base_model, checkpoint_path)

# Push your PEFT model (adapters)
model.push_to_hub("Saef/mistral_attention_second_epoch")

# # Push the tokenizer as well
tokenizer.push_to_hub("Saef/mistral_attention_second_epoch")

Loading checkpoint from checkpoints/mistral_attention/checkpoint-640...


/rhome/sfaa2021/miniconda3/envs/hf_env/lib/python3.10/site-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/602M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Saef/mistral_attention_second_epoch/commit/30cb506b0cee1d401423d3267f882dcc5c65f466', commit_message='Upload tokenizer', commit_description='', oid='30cb506b0cee1d401423d3267f882dcc5c65f466', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Saef/mistral_attention_second_epoch', endpoint='https://huggingface.co', repo_type='model', repo_id='Saef/mistral_attention_second_epoch'), pr_revision=None, pr_num=None)